In [2]:
import torch

### Comparison

#### CNN
+ 权重共享 平移不变性、可并行计算
+ 滑动窗口 局部关联性建模、依靠多层堆积来进行长程建模
+ 对相对位置敏感、对绝对位置不敏感

#### RNN 依次有序递归建模
+ 对顺序敏感
+ 串行计算耗时
+ 长程建模能力弱
+ 计算复杂度与序列长度呈线性关系
+ 单步计算复杂度不变
+ 对相对位置敏感，对绝对位置敏感
  
#### tranformer
+ 无局部假设
  + 可并行计算
  + 对相对位置不敏感
+ 无有序假设
  + 需要位置编码反映位置变化对特征的影响
  + 对绝对位置不敏感
+ 任意两字符都可以建模
  + 擅长长短程建模
  + 自注意力机制需要序列长度的平方级复杂运算

### Transformer encoder
+ input word embedding
  由稀疏的one-hot进入一个不带bias的FFN得到一个稠密的连续箱梁
+ position encoding
  + 通过sin/cos来固定表征
    + 每个位置确定
    + 对于不同的句子，相同位置的距离一致
    + 可以推广到更长的测试句子
  + pe(pos+k)可以写成pe(k)的线性组合
  + 通过残差连接来是的位置信息流入深层
+ multi-head self-attention
  + 建模能力更强，表征空间更丰富
  + 由多组Q\K\V构成，每组单独计算一个attention向量
  + 把每组的attention向量拼起来，并进入一个FFN中得到最终的向量
+ feed-forward network
  + 只考虑每个单独位置进行建模
  + 不同位置参数共享
  + 类似于1*1 pointwise convolution

### Transformer Decoder
+ output word embedding
+ masked multi-head self-attention
+ multi-head cross-attention
+ feed-forward network
+ softmax

### 自回归模型
+ word embedding
+ position embeding
+ encoder self-attention mask
+ intra-attention mask
+ decoder self-attention mask
+ multi-head self-attention

In [10]:
import torch
import numpy
import torch.nn as nn
import torch.nn.functional as F

# word embedding, 以序列建模为例
# 考虑source sentence target sentence
# 这里考虑离散的建模
# 构建序列，序列字符以其在词表索引形式表示

batch_size = 2
src_len = torch.randint(2,5,(batch_size,))
tgt_len = torch.randint(2,5,(batch_size,))
# 生成两个序列，每个序列都有一个随机生成的长度，最小为2，最大为5
# src_seq
# tgt_seq
print(src_len)
print(tgt_len)

tensor([4, 4])
tensor([4, 4])


In [36]:
# 序列单词总数，单词表大小
max_num_src_words = 8
max_num_tgt_words = 8
#  序列最大程度
max_src_seq_len = 5
max_tgt_seq_len = 5

src_len = torch.Tensor([2,4]).to(torch.int32)
tgt_len = torch.Tensor([4,3]).to(torch.int32)

# 以单词索引构成的句子
# src_seq = torch.cat([torch.unsequeeze(F.pad(torch.randint(1,max_num_src_words,(L,)),(0,max_src_seq_len-L)),0) \ 
#                     for L in src_len ])
src_seq = [ F.pad(torch.randint(1,max_num_src_words,(L,)),(0,max_src_seq_len-L)) for L in src_len ]

print(src_seq)
# 71 1364
tgt_seq = [ F.pad(torch.randint(1,max_num_tgt_words,(L,)),(0,max_src_seq_len-L)) for L in tgt_len ]
print(tgt_seq)

[tensor([4, 2, 0, 0, 0]), tensor([2, 7, 5, 4, 0])]
[tensor([1, 1, 5, 6, 0]), tensor([6, 1, 7, 0, 0])]


In [37]:
# 对每个张量做pad,拼在一起
# torch.unsequeeze升维度
# src_seq = [torch.unsqueeze(T,dim=1) for T in src_seq]
# print(src_seq)
# torch.cat两个张量拼接，只能在已有的维度上，横向拼或者纵向拼
# 默认值为0
src_seq = torch.stack(tuple(src_seq),0)
print(src_seq)
tgt_seq = torch.stack(tuple(tgt_seq),0)
print(tgt_seq)

tensor([[4, 2, 0, 0, 0],
        [2, 7, 5, 4, 0]])
tensor([[1, 1, 5, 6, 0],
        [6, 1, 7, 0, 0]])


In [39]:
# 构造embedding
model_dim = 4

src_embedding_table = nn.Embedding(max_num_src_words+1,model_dim)
tgt_embedding_table = nn.Embedding(max_num_tgt_words+1,model_dim)
print(src_embedding_table.weight)

Parameter containing:
tensor([[ 7.2626e-04, -7.0833e-02, -9.3470e-01,  1.1986e+00],
        [-6.2423e-01, -1.4757e+00, -1.5339e-01,  5.2330e-01],
        [ 1.5762e-01,  1.0453e+00,  3.5325e-01, -8.5912e-01],
        [-6.6104e-01, -6.0316e-02, -1.2233e+00, -1.7766e+00],
        [ 2.0727e+00, -7.1832e-03, -4.9131e-01,  5.9102e-01],
        [-7.9353e-01,  1.2585e+00, -1.7252e+00,  1.3896e+00],
        [ 2.9879e-01, -1.4153e+00, -3.6494e-01,  5.6291e-01],
        [ 3.0750e-01,  7.2336e-02, -9.4862e-01, -7.6832e-01],
        [-6.4298e-01, -5.1511e-03,  8.5293e-01, -2.9075e-01]],
       requires_grad=True)


In [42]:
src_embedding = src_embedding_table(src_seq)
print(src_seq)
print(src_embedding)
# 迪奥用embedding的forward方法
tgt_embedding = tgt_embedding_table(tgt_seq)

tensor([[4, 2, 0, 0, 0],
        [2, 7, 5, 4, 0]])
tensor([[[ 2.0727e+00, -7.1832e-03, -4.9131e-01,  5.9102e-01],
         [ 1.5762e-01,  1.0453e+00,  3.5325e-01, -8.5912e-01],
         [ 7.2626e-04, -7.0833e-02, -9.3470e-01,  1.1986e+00],
         [ 7.2626e-04, -7.0833e-02, -9.3470e-01,  1.1986e+00],
         [ 7.2626e-04, -7.0833e-02, -9.3470e-01,  1.1986e+00]],

        [[ 1.5762e-01,  1.0453e+00,  3.5325e-01, -8.5912e-01],
         [ 3.0750e-01,  7.2336e-02, -9.4862e-01, -7.6832e-01],
         [-7.9353e-01,  1.2585e+00, -1.7252e+00,  1.3896e+00],
         [ 2.0727e+00, -7.1832e-03, -4.9131e-01,  5.9102e-01],
         [ 7.2626e-04, -7.0833e-02, -9.3470e-01,  1.1986e+00]]],
       grad_fn=<EmbeddingBackward>)
